In [ ]:
!pip install pdfkit --quiet
!apt-get install -y wkhtmltopdf --quiet
!pip install PyPDF2 --quiet
!pip install tqdm --quiet
!pip install llama-index llama-parse --quiet
!pip install llama-index-vector-stores-pinecone --quiet
!pip install PyMuPDF --quiet
!pip install requests beautifulsoup4 pdfkit  --quiet
!pip install llama-index-agent-openai --quiet
!pip install llama-index-llms-openai --quiet
!pip install llama-index --quiet

# Setup

In [ ]:
import os
import requests
import io
import pdfkit
import fitz  # PyMuPDF
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import random
import nest_asyncio
from llama_parse import LlamaParse
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, StorageContext, Document, SummaryIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SimpleNodeParser, SentenceSplitter
from llama_index.llms.openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-proj-ccOo3JXmy0uUY29D3pKyT3BlbkFJKuYzjzVkpD4tkEwkCQZM"

os.environ["PINECONE_API_KEY"] = "c0681ba1-c767-42fe-b5c7-b4fdd9c5c438"

api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=api_key)

# Loading

In [ ]:
# List of URLs to scrape
url_parts = [
    "8", "Aiah", "Anagolay_(Born_To_Win)", "BINI", "BINI/Gallery",
    "BINI:_The_Launch", "BINI_Wiki", "BLOOM_(Fandom)", "B_HU_U_R",
    "Born_To_Win", "Born_To_Win_(Album)", "Born_To_Win_(song)",
    "Born_To_Win_%3D_Bahasa_Indonesia_Version", "Born_To_Win_%3D_Japanese_Version",
    "Born_To_Win_%3D_Spanish_Version", "Born_To_Win_%3D_Thai_Version",
    "Born_To_Win_Maxi", "Colet", "Da_Coconut_Nut", "Feel_Good_(Album)",
    "G22", "Golden_Arrow", "Gwen", "Here_With_You", "Huwag_Muna_Tayong_Umuwi",
    "I_Feel_Good", "Jhoanna", "Kapit_Lang", "Karera", "Kinikilig",
    "Lagi", "Lian_Kyla", "List_of_BINI_Performances", "Main_Page", "Maloi",
    "Maloi/Gallery", "Mayari_(Born_To_Win)", "Mikha", "Na_Na_Na", "No_Fear",
    "One_Dream:_The_BINI-BGYO_Journey", "Pantropiko", "SAB", "Sheena",
    "Sodop_(Born_To_Win)", "Stacey", "Star_Hunt_Academy", "Star_Hunt_Trainee_TV",
    "Strings", "Strings_-_Dance_ver.", "Zero_World_(Born_To_Win)"
]

# Function to generate full URLs
def generate_urls(base_url, parts):
    return [f"{base_url}{part}" for part in parts]

# Base URL
base_url = "https://bini.fandom.com/wiki/"

# Generate full URLs
urls = generate_urls(base_url, url_parts)

In [ ]:
# Create a directory to save individual text files
os.makedirs("text_data", exist_ok=True)

# Function to extract text from a URL using BeautifulSoup
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main content area
    content_div = soup.find('div', {'class': 'mw-parser-output'})
    if content_div:
        text = content_div.get_text(separator='\n', strip=True)
        return text
    return ""

# Scrape text from each URL and save it to a text file, with a progress bar
text_files = []
for i, url in enumerate(tqdm(urls, desc="Scraping URLs for text")):
    text_filename = f"text_data/page_{i+1}.txt"
    text = extract_text_from_url(url)
    with open(text_filename, 'w', encoding='utf-8') as file:
        file.write(text)
    text_files.append(text_filename)

# Convert each text file to a PDF
os.makedirs("data", exist_ok=True)
pdf_files = []
for i, text_file in enumerate(tqdm(text_files, desc="Converting text to PDFs")):
    pdf_filename = f"data/page_{i+1}.pdf"
    pdfkit.from_file(text_file, pdf_filename)
    pdf_files.append(pdf_filename)

Scraping URLs for text:   0%|          | 0/51 [00:00<?, ?it/s]

Converting text to PDFs:   0%|          | 0/51 [00:00<?, ?it/s]

# Loading with LlamaParse

In [ ]:
# Apply nest_asyncio to allow nested event loops (useful in Jupyter/Colab notebooks)
nest_asyncio.apply()

# Initialize LlamaParse with necessary parameters
parser = LlamaParse(
    api_key="llx-V4Rrgehs2Y89rvxzhu7HGkwTH8P9NQnu7WmkovId3ow1UkiO",  # Replace with your actual API key
    result_type="text",  # "markdown" and "text" are available
    verbose=False,  # Set to False to reduce verbosity
    language="en",  # Optionally you can define a language, default=en
)

In [ ]:
# Define the directory containing the PDF files
data_directory = "/content/data"

# List all PDF files in the directory
document_paths = [os.path.join(data_directory, file) for file in os.listdir(data_directory) if file.endswith(".pdf")]

# Initialize an empty list to store the processed documents
processed_documents = []

# Process each document with a single progress bar
for document_path in tqdm(document_paths, desc="Processing documents"):
    document = parser.load_data([document_path])
    processed_documents.extend(document)

Processing documents:   0%|          | 0/51 [00:00<?, ?it/s]


Parsing files: 100%|██████████| 1/1 [00:19<00:00, 19.95s/it]

Parsing files: 100%|██████████| 1/1 [00:06<00:00,  6.26s/it]

Parsing files: 100%|██████████| 1/1 [00:12<00:00, 12.74s/it]

Parsing files: 100%|██████████| 1/1 [00:19<00:00, 19.07s/it]

Parsing files: 100%|██████████| 1/1 [00:10<00:00, 10.59s/it]

Parsing files: 100%|██████████| 1/1 [00:21<00:00, 21.58s/it]

Parsing files: 100%|██████████| 1/1 [00:04<00:00,  4.27s/it]

Parsing files: 100%|██████████| 1/1 [00:06<00:00,  6.26s/it]

Parsing files: 100%|██████████| 1/1 [00:21<00:00, 21.49s/it]

Parsing files: 100%|██████████| 1/1 [00:17<00:00, 17.00s/it]

Parsing files: 100%|██████████| 1/1 [00:04<00:00,  4.04s/it]

Parsing files: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it]

Parsing files: 100%|██████████| 1/1 [00:19<00:00, 19.23s/it]

Parsing files: 100%|██████████| 1/1 [00:32<00:00, 32.02s/it]

Parsing files: 100%|██████████| 1/1 [00:14<00:00, 14.90s/it]

Parsing files: 100%|██████████| 1/1 [00:12<00:00, 12.69s/it]

Parsing

In [ ]:
# Display the total number of documents
total_documents = len(processed_documents)
print(f"Total number of documents: {total_documents}")

# Randomly select one document to print
random_index = random.randint(0, total_documents - 1)
print(f"Randomly selected document {random_index + 1}:\n")
display(processed_documents[random_index])

Total number of documents: 51
Randomly selected document 27:



Document(id_='93e15b45-89ff-4ae9-b3ab-265ff9347b7c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='BLOOM (Fandom)\nType\nFandom\nFounded\nFebruary 13, 2021\nFounder\nBINI\nLocation\nPhilippines\nBLOOM\n(stylized as\nBLâˆžM\n) is the official fandom name for the Filipino girl group BINI.\nName\n[\n]\nThe word Bloom, defined as the blossoming state of a flower, was coined by a fan with the meaning, "A lovely reminder of\nBINI\'s story and journey. It stands for the beginning of springtime, that there\'s always beauty after every cold trial."\nThe\nâˆž\n(Infinity) sign incorporated in the fandom name is BINI\'s official hand sign, symbolizing the number "8", which is the number\nof BINI members and inspired their chant\n"walo hanggang dulo"\n(eight until the end).\nFounding\n[\n]\nOn February 13, 2021, BINI had a livestream on Facebook with the caption "BINI Special Announcement". In the live the members\nvoted for the

# Preparing Storing

In [ ]:
node_parser = SimpleNodeParser()

nodes = node_parser.get_nodes_from_documents(processed_documents)

In [ ]:
# Print the number of nodes parsed
print(f"Number of nodes parsed: {len(nodes)}")

# Print details of the first few nodes to inspect their structure
for i, node in enumerate(nodes[:3]):  # Adjust the range if needed
    print(f"\nNode {i} details:")
    display(node)

Number of nodes parsed: 88

Node 0 details:


TextNode(id_='837b8c85-34f6-4102-b7cb-4bfe06552c53', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='31c4d0ec-bfa0-4e09-98f1-9cdf24a7d9e2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='851c21589e01def141b1e08772fba782655761285ccfa710dd7c2b54f7c336c3'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8f12b855-0ead-4fa1-88f9-4e838c209e7e', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='5c213ed0859ecc33c73a43c296c51add0c2e2edd0be654c8a58d07c2906c2dab')}, text='Golden Arrow\nArtist(s)\nBINI\nReleased\nOctober 12, 2021\nGenre\nOPM, EDM\nLength\n3:47\nLyrics\nSabine Cerrado a.k.a\nSAB\nComposer\nSabine Cerrado a.k.a\nSAB\nArranger\nTheo Martel\nProducer\nJonathan Manalo\nStar Music\n"Golden Arrow"\nis the fourth official single of\nBINI\n. It was released on October 14, 2021 at 07:00 pm PHT as the carrier single of their first album titled\n"\nBorn To 


Node 1 details:


TextNode(id_='8f12b855-0ead-4fa1-88f9-4e838c209e7e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='31c4d0ec-bfa0-4e09-98f1-9cdf24a7d9e2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='851c21589e01def141b1e08772fba782655761285ccfa710dd7c2b54f7c336c3'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='837b8c85-34f6-4102-b7cb-4bfe06552c53', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1b7aba2c72ac3a02cf9b1066625b6a2a82e2a82b6a46f11ba8341db5b27caacc'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2dcd046a-ca3d-4d42-b279-b7a3566e0caa', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='20c080b1fc1720c031c43adf730bc2fe51b6ff9ffb1c3d0c04603ae6fc48617c')}, text='The vocal arrangement were done by Anna Achacoso-Graham & Jonathan Manalo.\nContents\n1\nBackground and Release\n2\nAudio\n2.1\nSpotify\n2.2\nYoutube\n3\nLyrics\n4\nVideos\n5\nC


Node 2 details:


TextNode(id_='2dcd046a-ca3d-4d42-b279-b7a3566e0caa', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='31c4d0ec-bfa0-4e09-98f1-9cdf24a7d9e2', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='851c21589e01def141b1e08772fba782655761285ccfa710dd7c2b54f7c336c3'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='8f12b855-0ead-4fa1-88f9-4e838c209e7e', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='5c213ed0859ecc33c73a43c296c51add0c2e2edd0be654c8a58d07c2906c2dab')}, text='Nicolette Vergara\nArranged by Theo Martel\nMixed & Mastered by Tim Recla\nAll Vocals Recorded & Produced by: Jonathan Manalo at The Purple Room\nVocal Arrangement by: Jonathan Manalo and Anna Achacoso-Graham\nOver-all Production by: Jonathan Manalo\nSTAR MUSIC AND MARKETING TEAM\nStar Music Digital Production Team\nDigital Content Producer: Jaliza Baluyut\nDigital Support Specialist: Ice 

In [ ]:
# Function to create an index if it doesn't exist
def create_index_if_not_exists(index_name, dimension, metric, spec):
    existing_indexes = pc.list_indexes()
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric=metric,
            spec=spec
        )
        print(f"Index '{index_name}' created successfully.")
    else:
        print(f"Index '{index_name}' already exists.")

# Usage
create_index_if_not_exists(
    index_name="bini_rag",
    dimension=1536,
    metric="dotproduct",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '08bbfbf9347f6fb78f8ed42374e5520b', 'Date': 'Fri, 17 May 2024 15:42:40 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
pinecone_index = pc.Index("bini_rag")

In [ ]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)

# Indexing

In [ ]:
# Specify your OpenAI API key and the embedding model you want to use
api_key = "sk-proj-ccOo3JXmy0uUY29D3pKyT3BlbkFJKuYzjzVkpD4tkEwkCQZM"
embedding_model = "text-embedding-3-small"  # Replace with the actual model name

# Initialize the OpenAIEmbedding with the specified model
embed_model = OpenAIEmbedding(api_key=api_key, model=embedding_model)

# Assume astra_db_store is already initialized
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
# Initialize the VectorStoreIndex with the nodes and storage context
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    embed_model=embed_model,
)

In [ ]:
# Create the VectorStoreIndex using the existing Pinecone collection
index = VectorStoreIndex.from_vector_store(
    vector_store,
)

# Querying

In [ ]:
# Set up the query engine
query_engine = index.as_query_engine(vector_store_query_mode="hybrid")

In [ ]:
# Example query
response = query_engine.query("What is Colet's full name?")
print(response)